<a href="https://colab.research.google.com/github/sindla97/RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aproach
1. import pdf files to the knowledge base
2. Perfrom semantic segmentation using a sentense transfromer type model and convert them to embeddings
3. use a rerank model to rank the retrived embeddings
4. Use the LLM to generate response based on the query and retrvied emebeddings


Try to provide an option to update the knowlege base when new documents are provided,

In [2]:
!pip install langchain-google-genai pypdf langchain_experimental langchain-pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.4/441.4 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.7/367.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.1/

In [2]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = "/content/AI_Agent_white_paper_by_google_1737132048.pdf"
loader = PyPDFLoader(pdf_path)

In [3]:
all_pages=loader.load()
# we can use load_and_split() but we have to provide the textsplitter criterion if not it uses Recrusivetextsplitter by default
all_pages[10].page_content

'Agents\n11\nSeptember 2024\nd. Action input: The model’s decision on what inputs to provide to the tool (if any)\ne. Observation: The result of the action / action input sequence\ni. This thought / action / action input / observation could repeat N-times as needed\nf. Final answer: The model’s final answer to provide to the original user query\n4. The ReAct loop concludes and a final answer is provided back to the user\nFigure 2. Example agent with ReAct reasoning in the orchestration layer\nAs shown in Figure 2, the model, tools, and agent configuration work together to provide \na grounded, concise response back to the user based on the user’s original query. While \nthe model could have guessed at an answer (hallucinated) based on its prior knowledge, \nit instead used a tool (Flights) to search for real-time external information. This additional \ninformation was provided to the model, allowing it to make a more informed decision based \non real factual data and to summarize this 

In [4]:
from google.colab import userdata
google_api_key=userdata.get('GOOGLE_API_KEY')


In [5]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

# Initialize the text splitter
text_splitter = SemanticChunker(GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key))
chunks = text_splitter.create_documents([''.join([page.page_content for page in all_pages])]) # create document takes in list of dcoumetns and splits it, we have to pass the whole document as string

In [15]:
chunks[0].page_content

'Agents\nAuthors: Julia Wiesinger, Patrick Marlow  \nand Vladimir VuskovicAgents\n2\nSeptember 2024\nAcknowledgements\nReviewers and Contributors\nEvan Huang\nEmily Xue\nOlcan Sercinoglu\nSebastian Riedel\nSatinder Baveja\nAntonio Gulli\nAnant Nawalgaria\nCurators and Editors\nAntonio Gulli\nAnant Nawalgaria\nGrace Mollison \nTechnical Writer\nJoey Haymaker\nDesigner\nMichael LanningIntroduction 4\nWhat is an agent? 5\n The model 6\n The tools 7\n The orchestration layer 7\n Agents vs. models 8\n Cognitive architectures: How agents operate  8\nTools: Our keys to the outside world 12\n Extensions  13\n  Sample Extensions  15\n Functions  18\n  Use cases 21\n  Function sample code 24\n Data stores 27\n  Implementation and application 28\n Tools recap 32\nEnhancing model performance with targeted learning 33\nAgent quick start with LangChain 35\nProduction applications with Vertex AI agents 38\nSummary 40\nEndnotes 42\nTable of contentsAgents\n4\nSeptember 2024\nIntroduction\nHumans are f

In [14]:
len(chunks)

18

In [6]:
k=GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key).embed_documents([ x.page_content for x in chunks])
# there are two options to generate embedding on is using embed_documents( which takes in list of strings) & embed_query(takes a string and converts to embeddings)

In [8]:
# lets use a vector store  to store these embeddings
from langchain_pinecone.vectorstores import PineconeVectorStore
vecstore=PineconeVectorStore(embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key),pinecone_api_key=userdata.get('pinecone_api'), index_name='rag-test')
vecstore

In [9]:
vecstore.add_documents(chunks)

['dd069f50-3ea4-43b2-a167-043e83cf958f',
 '59dfa83a-723d-43a1-bf8a-e81bb015b475',
 'dee5fe7e-02f1-43a0-8d8e-5d18598e434d',
 'd8426ee9-29cc-4235-ae64-56bdc96cafbb',
 'f2060c27-0949-4721-a294-914dcec8526a',
 'cdb17c59-9486-41f0-8ba0-427381acaea5',
 '206825cc-4743-417f-aa09-42cab29dd84d',
 '55636f5b-5596-4e3a-8add-9dcaa1d289a5',
 '14f6e346-987e-4dac-b21b-65b797c3e0c7',
 '6f274d2e-d054-4a4a-b17d-a10cd934b6e1',
 '10046cdb-15f6-4869-91bb-e5221c2df26a',
 '57cfe15b-1227-42f7-bc6f-ad81885ffce0',
 'f5b75702-52ba-4c42-9c8d-09dcd04a8c97',
 '5503f731-1682-4156-bcec-37249a939281',
 '671577da-39bf-4be9-a47a-7029d6adb356',
 'a4c8be95-5fa6-4593-abfa-4da42159b1e5',
 '8075956f-5be1-413f-9030-742f127c1871',
 '1e3cee1f-c08b-40de-b0a8-153cbf0029e4']

In [11]:
vecstore.search(query='what are the contents of the document Agentic AI', search_type='similarity')

[Document(id='8075956f-5be1-413f-9030-742f127c1871', metadata={}, page_content="6. Yao, S. et al., 2023, 'Tree of Thoughts: Deliberate Problem Solving with Large Language Models'. Available at:  \nhttps://arxiv.org/abs/2305.10601 . 7. Long, X., 2023, 'Large Language Model Guided Tree-of-Thought'. Available at:  \nhttps://arxiv.org/abs/2305.08291 ."),
 Document(id='5503f731-1682-4156-bcec-37249a939281', metadata={}, page_content="Diao, S. et al., 2023, 'Active Prompting with Chain-of-Thought for Large Language Models'. Available at:  \nhttps://arxiv.org/pdf/2302.12246.pdf ."),
 Document(id='dd069f50-3ea4-43b2-a167-043e83cf958f', metadata={}, page_content='Agents\nAuthors: Julia Wiesinger, Patrick Marlow  \nand Vladimir VuskovicAgents\n2\nSeptember 2024\nAcknowledgements\nReviewers and Contributors\nEvan Huang\nEmily Xue\nOlcan Sercinoglu\nSebastian Riedel\nSatinder Baveja\nAntonio Gulli\nAnant Nawalgaria\nCurators and Editors\nAntonio Gulli\nAnant Nawalgaria\nGrace Mollison \nTechnical 

In [12]:
vecstore.search(query='what is the summary of the document Agentic AI', search_type='similarity', k=4)

[Document(id='8075956f-5be1-413f-9030-742f127c1871', metadata={}, page_content="6. Yao, S. et al., 2023, 'Tree of Thoughts: Deliberate Problem Solving with Large Language Models'. Available at:  \nhttps://arxiv.org/abs/2305.10601 . 7. Long, X., 2023, 'Large Language Model Guided Tree-of-Thought'. Available at:  \nhttps://arxiv.org/abs/2305.08291 ."),
 Document(id='10046cdb-15f6-4869-91bb-e5221c2df26a', metadata={}, page_content='3.'),
 Document(id='5503f731-1682-4156-bcec-37249a939281', metadata={}, page_content="Diao, S. et al., 2023, 'Active Prompting with Chain-of-Thought for Large Language Models'. Available at:  \nhttps://arxiv.org/pdf/2302.12246.pdf ."),
 Document(id='f5b75702-52ba-4c42-9c8d-09dcd04a8c97', metadata={}, page_content='4.')]

In [13]:
# send the retrived documents to llm and ouptut in a structured format
from langchain_google_genai import ChatGoogleGenerativeAI

output_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001", google_api_key=google_api_key)
#llm.invoke("Write me a ballad about LangChain")
#lets build a message


In [17]:
user_query='what is the summary of the document Agentic AI'
retrived_contents=vecstore.search(query=user_query, search_type='similarity', k=4)
message=f"Based on the contents provided {''.join([x.page_content for x in retrived_contents])} answer the user question:{user_query} in a structured format and ask a followup question"
output_llm.invoke(message).content

'Based on the provided document snippets, there is no mention of "Agentic AI". Therefore, I cannot provide a summary of it.\n\nHowever, I can tell you that the documents discuss techniques for improving the problem-solving capabilities of Large Language Models (LLMs). These techniques include:\n\n*   **Tree of Thoughts (ToT):** A method that allows LLMs to explore multiple reasoning paths and evaluate different options to solve complex problems.\n*   **Large Language Model Guided Tree-of-Thought:** Likely a variation or application of the ToT approach, possibly with specific guidance mechanisms.\n*   **Active Prompting with Chain-of-Thought:** A technique that uses targeted prompts to guide LLMs through a reasoning process, encouraging them to break down problems into smaller, more manageable steps.\n\nSince I can\'t summarize "Agentic AI" based on the provided context, I will ask a follow-up question related to the techniques that *are* mentioned:\n\n**Follow-up question:** How do Tre

In [18]:
def chat(user_query):
  retrived_contents=vecstore.search(query=user_query, search_type='similarity', k=4)
  message=f"Based on the contents provided {''.join([x.page_content for x in retrived_contents])} answer the user question:{user_query} in a structured format and ask a followup question"
  print(output_llm.invoke(message).content)
  new_user_query=input()
  if len(new_user_query)>0:
    chat(new_user_query)

chat('what are the contents of the document Agentic AI')

Okay, based on the provided text, here's a structured summary of the document "Agents" (dated September 2024):

**Document Title:** Agents

**Authors:** Julia Wiesinger, Patrick Marlow, Vladimir Vuskovic

**Acknowledgements (Reviewers and Contributors):** Evan Huang, Emily Xue, Olcan Sercinoglu, Sebastian Riedel, Satinder Baveja, Antonio Gulli, Anant Nawalgaria

**Curators and Editors:** Antonio Gulli, Anant Nawalgaria, Grace Mollison

**Technical Writer:** Joey Haymaker

**Designer:** Michael Lanning

**Table of Contents/Sections:**

*   Introduction
*   What is an agent?
    *   The model
    *   The tools
    *   The orchestration layer
    *   Agents vs. models
    *   Cognitive architectures: How agents operate
*   Tools: Our keys to the outside world
    *   Extensions
        *   Sample Extensions
    *   Functions
        *   Use cases
        *   Function sample code
    *   Data stores
        *   Implementation and application
    *   Tools recap
*   Enhancing model performa

KeyboardInterrupt: Interrupted by user